In [1]:
from datetime import datetime, timedelta
import io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import statsmodels.api as sm

from utils.loading import read_ticker, \
    read_data_from_tar,\
    read_multiple,\
    clean_extracts
from utils.loading import \
    DATA_DIR_PATH

CA_TRADE_PATH = os.path.join(DATA_DIR_PATH, 'trade', 'AAB.TO')

In [2]:
df = read_ticker(CA_TRADE_PATH)

### Preprocessing

In [8]:
def xltime_to_datetime(xltime):
    excel_epoch = datetime(1899, 12, 30)
    return excel_epoch + timedelta(days=xltime)

def extract_ids(s, id_type):
    pattern = rf"\[{id_type}\](\d+)"
    match = re.search(pattern, s)
    return int(match.group(1)) if match else None

In [7]:
extract_ids("AAB.TO [ID1234]", "ID")

1234

In [93]:
df['datetime'] = df['xltime'].apply(xltime_to_datetime)

df['seller_id'] = df['trade-rawflag'].apply(lambda x: extract_ids(x, 'BNPP Seller ID'))
df['buyer_id'] = df['trade-rawflag'].apply(lambda x: extract_ids(x, 'BNPP Buyer ID'))

df['trade-stringflag'] = df['trade-stringflag'].astype('category')

df_cleaned = df[['datetime', 'trade-price', 'trade-volume', 'seller_id', 'buyer_id']].copy()

In [118]:
daily_volumes_std = df_cleaned.groupby(df_cleaned['datetime'].dt.date)\
    .agg({'trade-volume': 'sum',
          'trade-price': 'std'})\
    .reset_index()\
    .rename(columns={'datetime': 'day',
                     'trade-volume': 'day_volume',
                     'trade-price': 'day_price_std'})
daily_volumes_std['day'] = pd.to_datetime(daily_volumes_std['day'])

df_cleaned['day'] = df_cleaned.datetime.dt.date
df_cleaned['day'] = pd.to_datetime(df_cleaned['day'])

In [119]:
daily_volumes_std.dtypes

day              datetime64[ns]
day_volume              float64
day_price_std           float64
dtype: object

In [127]:
df_cleaned_plus = pd.merge(df_cleaned,
         daily_volumes_std.reset_index()\
            .rename(columns={'datetime': 'day'}),
         on='day')[['datetime', 'trade-price', 'trade-volume', 'seller_id', 'buyer_id', 'day_volume', 'day_price_std']]
df_cleaned_plus.set_index('datetime', inplace=True)
df_cleaned_plus.sort_index(inplace=True)

df_cleaned_plus.head()

,trade-price,trade-volume,seller_id,buyer_id,day_volume,day_price_std
datetime,,,,,,
2016-01-04 14:30:00.098,0.160,2000.0,79,7,397543.0,0.008399
2016-01-04 14:33:28.094,0.160,5500.0,79,74,397543.0,0.008399
2016-01-04 14:39:28.567,0.165,10000.0,2,58,397543.0,0.008399
2016-01-04 14:43:50.170,0.160,3000.0,2,74,397543.0,0.008399
2016-01-04 15:45:06.853,0.160,3000.0,2,58,397543.0,0.008399


In [178]:
# sample a day
df_cleaned_plus[df_cleaned_plus.index.date == datetime(2016, 7, 27).date()].sort_values('buyer_id', ascending=False)

,trade-price,trade-volume,seller_id,buyer_id,day_volume,day_price_std
datetime,,,,,,
2016-07-27 19:22:32.645000,0.165,7000.0,7,7,19500.0,0.010408
2016-07-27 14:43:58.507000,0.145,1500.0,15,2,19500.0,0.010408
2016-07-27 19:04:53.646001,0.150,11000.0,7,2,19500.0,0.010408


#### GPT Code

In [40]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Assuming df is your preprocessed DataFrame

# Aggregate data
aggregated_data = df_cleaned.resample('min').agg({
    'trade-volume': 'sum',
    'trade-price': lambda x: np.average(x, weights=df_cleaned.loc[x.index, 'trade-volume'])  # Weighted average
})

# Calculate price changes
aggregated_data['price_change'] = aggregated_data['trade-price'].diff().abs()

# Square root of trade volume
aggregated_data['sqrt_volume'] = np.sqrt(aggregated_data['trade-volume'])

# Linear regression
X = sm.add_constant(aggregated_data['sqrt_volume'])  # Add constant term
y = aggregated_data['price_change']
model = sm.OLS(y, X, missing='drop').fit()  # Drop NaN values

# Plotting
plt.scatter(aggregated_data['sqrt_volume'], aggregated_data['price_change'])
plt.plot(aggregated_data['sqrt_volume'], model.predict(X), color='red')  # Regression line
plt.xlabel('Square Root of Trade Volume')
plt.ylabel('Absolute Price Change')
plt.show()

# Output model summary
print(model.summary())


ZeroDivisionError: Weights sum to zero, can't be normalized